In [17]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import numpy as np
import random

In [8]:
model = DiscreteBayesianNetwork([
    ('Burglary', 'Alarm'),
    ('Earthquake', 'Alarm'),
    ('Alarm', 'JohnCalls'),
    ('Alarm', 'MaryCalls'),
])

# P(Burglary)
cpd_burglary = TabularCPD(variable='Burglary', variable_card=2, values=[[0.999], [0.001]])

# P(Earthquake)
cpd_earthquake = TabularCPD(variable='Earthquake', variable_card=2, values=[[0.998], [0.002]])

# P(Alarm | Burglary, Earthquake)
cpd_alarm = TabularCPD(
    variable='Alarm', 
    variable_card=2, 
    values=[
        [0.999, 0.71, 0.06, 0.05], # False
        [0.001, 0.29, 0.94, 0.95], # True
    ],
    evidence=['Burglary', 'Earthquake'],
    evidence_card=[2, 2]
)

#  P(JohnCalls)
cpd_john = TabularCPD(variable='JohnCalls', variable_card=2, values=[[0.3, 0.9], [0.7, 0.1]], evidence=['Alarm'], evidence_card=[2])

cpd_mary = TabularCPD(variable='MaryCalls', variable_card=2, values=[[0.2, 0.99], [0.8, 0.01]], evidence=['Alarm'], evidence_card=[2])

model.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_john, cpd_mary)

assert model.check_model(), 'Model is incorrect'

inference = VariableElimination(model)

result = inference.query(variables=['Burglary'], evidence={'JohnCalls': 1, 'MaryCalls': 1})
print(result)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.9999 |
+-------------+-----------------+
| Burglary(1) |          0.0001 |
+-------------+-----------------+


# Task 1

In [10]:
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']

deck = [(rank, suit) for suit in suits for rank in ranks]

total_cards = len(deck)

red_cards = [card for card in deck if card[1] in ['Hearts', 'Diamonds']]
p_red = len(red_cards) / total_cards

heart_cards = [card for card in red_cards if card[1] == 'Hearts']
p_heart_given_red = len(heart_cards) / len(red_cards)

face_ranks = ['Jack', 'Queen', 'King']
face_cards = [card for card in deck if card[0] in face_ranks]
diamond_face_cards = [card for card in face_cards if card[1] == 'Diamonds']
p_diamond_given_face = len(diamond_face_cards) / len(face_cards)

spades_among_face = [card for card in face_cards if card[1] == 'Spades']
queens_among_face = [card for card in face_cards if card[0] == 'Queen']
union_spade_or_queen = set(spades_among_face + queens_among_face)
p_spade_or_queen_given_face = len(union_spade_or_queen) / len(face_cards)

print(f"1. P(Red card): {p_red:.2f}")
print(f"2. P(Heart | Red card): {p_heart_given_red:.2f}")
print(f"3. P(Diamond | Face card): {p_diamond_given_face:.2f}")
print(f"4. P(Spade or Queen | Face card): {p_spade_or_queen_given_face:.2f}")

1. P(Red card): 0.50
2. P(Heart | Red card): 0.50
3. P(Diamond | Face card): 0.25
4. P(Spade or Queen | Face card): 0.50


# Task 2

In [15]:
model = DiscreteBayesianNetwork([
    ('Intelligence', 'Grade'),
    ('StudyHours', 'Grade'),
    ('Difficulty', 'Grade'),
    ('Grade', 'Pass')
])

# P(Intelligence)
cpd_intelligence = TabularCPD(variable='Intelligence', variable_card=2, values=[[0.7], [0.3]]) # High, Low

# P(StudyHours)
cpd_study_hours = TabularCPD(variable='StudyHours', variable_card=2, values=[[0.6], [0.4]]) # Sufficient, Insufficient

# P(Difficulty)
cpd_difficulty = TabularCPD(variable='Difficulty', variable_card=2, values=[[0.4], [0.6]]) # Hard, Easy

# P(Grade | Intelligence, StudyHours, Difficulty)
# I=High(0)/Low(1), S=Sufficient(0)/Insufficient(1), D=Hard(0)/Easy(1)
cpd_grade = TabularCPD(
    variable='Grade', variable_card=3,
    values=[
        # Grade=A
        [0.9, 0.7, 0.8, 0.6, 0.4, 0.3, 0.2, 0.1],  
        # Grade=B
        [0.08, 0.2, 0.15, 0.25, 0.3, 0.4, 0.3, 0.2],  
        # Grade=C
        [0.02, 0.1, 0.05, 0.15, 0.3, 0.3, 0.5, 0.7]  ,
    ],
    evidence=['Intelligence', 'StudyHours', 'Difficulty'],
    evidence_card=[2, 2, 2],
)

# P(Pass | Grade)
cpd_pass = TabularCPD(
    variable='Pass', variable_card=2,
    values=[
        [0.95, 0.80, 0.50],
        [0.05, 0.20, 0.50]
    ],
    evidence=['Grade'],
    evidence_card=[3]
)

model.add_cpds(cpd_intelligence, cpd_study_hours, cpd_difficulty, cpd_grade, cpd_pass)

assert model.check_model(), 'Model is incorrect'

inference = VariableElimination(model)

result1 = inference.query(variables=['Pass'], evidence={'StudyHours': 0, 'Difficulty': 0})
print("P(Pass | StudyHours=Sufficient, Difficulty=Hard):\n", result1)

result2 = inference.query(variables=['Intelligence'], evidence={'Pass': 0})
print("\nP(Intelligence | Pass=Yes):\n", result2)

P(Pass | StudyHours=Sufficient, Difficulty=Hard):
 +---------+-------------+
| Pass    |   phi(Pass) |
+=========+=============+
| Pass(0) |      0.8813 |
+---------+-------------+
| Pass(1) |      0.1187 |
+---------+-------------+

P(Intelligence | Pass=Yes):
 +-----------------+---------------------+
| Intelligence    |   phi(Intelligence) |
+=================+=====================+
| Intelligence(0) |              0.7441 |
+-----------------+---------------------+
| Intelligence(1) |              0.2559 |
+-----------------+---------------------+


# Task 3

In [16]:
model = DiscreteBayesianNetwork([
    ('Disease', 'Fever'),
    ('Disease', 'Cough'),
    ('Disease', 'Fatigue'),
    ('Disease', 'Chills'),
])

# P(Disease)
cpd_disease = TabularCPD(variable='Disease', variable_card=2, values=[[0.3], [0.7]], state_names={'Disease': ['Flu', 'Cold']}) # Flu, Cold

# P(Fever, Disease)
cpd_fever = TabularCPD(
    variable='Fever', variable_card=2,
    values=[
        [0.9, 0.5], # Yes
        [0.1, 0.5], # No
    ],
    evidence=['Disease'],
    evidence_card=[2],
    state_names={'Fever': ['Yes', 'No'], 'Disease': ['Flu', 'Cold']}
)

# P(Cough, Disease)
cpd_cough = TabularCPD(
    variable='Cough', variable_card=2,
    values=[
        [0.8, 0.6], # Yes
        [0.2, 0.4], # No
    ],
    evidence=['Disease'],
    evidence_card=[2],
    state_names={'Cough': ['Yes', 'No'], 'Disease': ['Flu', 'Cold']}
)

# P(Fatigue, Disease)
cpd_fatigue = TabularCPD(
    variable='Fatigue', variable_card=2,
    values=[
        [0.7, 0.3], # Yes
        [0.3, 0.7], # No
    ],
    evidence=['Disease'],
    evidence_card=[2],
    state_names={'Fatigue': ['Yes', 'No'], 'Disease': ['Flu', 'Cold']}
)

cpd_chills = TabularCPD(
    variable='Chills', variable_card=2,
    values=[
        [0.6, 0.4], # Yes
        [0.4, 0.6], # No
    ],
    evidence=['Disease'],
    evidence_card=[2],
    state_names={'Chills': ['Yes', 'No'], 'Disease': ['Flu', 'Cold']}
)

model.add_cpds(cpd_disease, cpd_fever, cpd_cough, cpd_fatigue, cpd_chills)

assert model.check_model(), 'Model is incorrect'

inference = VariableElimination(model)

print("\nTask 1: P(Disease | Fever=Yes, Cough=Yes)")
print(inference.query(variables=['Disease'], evidence={'Fever': 'Yes', 'Cough': 'Yes'}))

print("\nTask 2: P(Disease | Fever=Yes, Cough=Yes, Chills=Yes)")
print(inference.query(variables=['Disease'], evidence={'Fever': 'Yes', 'Cough': 'Yes', 'Chills': 'Yes'}))

print("\nTask 3: P(Fatigue=Yes | Disease=Flu)")
print(inference.query(variables=['Fatigue'], evidence={'Disease': 'Flu'}))


Task 1: P(Disease | Fever=Yes, Cough=Yes)
+---------------+----------------+
| Disease       |   phi(Disease) |
+===============+================+
| Disease(Flu)  |         0.5070 |
+---------------+----------------+
| Disease(Cold) |         0.4930 |
+---------------+----------------+

Task 2: P(Disease | Fever=Yes, Cough=Yes, Chills=Yes)
+---------------+----------------+
| Disease       |   phi(Disease) |
+===============+================+
| Disease(Flu)  |         0.6067 |
+---------------+----------------+
| Disease(Cold) |         0.3933 |
+---------------+----------------+

Task 3: P(Fatigue=Yes | Disease=Flu)
+--------------+----------------+
| Fatigue      |   phi(Fatigue) |
+==============+================+
| Fatigue(Yes) |         0.7000 |
+--------------+----------------+
| Fatigue(No)  |         0.3000 |
+--------------+----------------+


# Task 4

In [18]:
states = ['Sunny', 'Cloudy', 'Rainy']

transition_matrix = [
    [0.6, 0.3, 0.1], 
    [0.3, 0.4, 0.3], 
    [0.2, 0.4, 0.4], 
]

def simulate_weather(start_state, days, transition_matrix, states):
    current_state = states.index(start_state)
    weather_sequence = [start_state]
    
    for _ in range(days - 1):
        next_state = np.random.choice(states, p=transition_matrix[current_state])
        weather_sequence.append(next_state)
        current_state = states.index(next_state)
    
    return weather_sequence

np.random.seed(0) 
weather_10_days = simulate_weather('Sunny', 10, transition_matrix, states)
print("Simulated 10-day weather sequence:", weather_10_days)

def estimate_rainy_probability(runs=10000):
    rainy_count = 0
    
    for _ in range(runs):
        sequence = simulate_weather('Sunny', 10, transition_matrix, states)
        if sequence.count('Rainy') >= 3:
            rainy_count += 1
    
    return rainy_count / runs

probability = estimate_rainy_probability(10000)
print("Estimated probability of at least 3 rainy days in 10 days:", round(probability, 4))

Simulated 10-day weather sequence: ['Sunny', np.str_('Sunny'), np.str_('Cloudy'), np.str_('Cloudy'), np.str_('Cloudy'), np.str_('Cloudy'), np.str_('Cloudy'), np.str_('Cloudy'), np.str_('Rainy'), np.str_('Rainy')]
Estimated probability of at least 3 rainy days in 10 days: 0.3295
